# Bayesian Optimization

We implement a Bayesian optimization for LED configurations that most closely match a target
light spectrum. We will work with a helper class, `SelfDrivingLabDemo` for setting up the
spectrophotometer sensor, measuring data from the sensor,
generating random inputs, and measuring the objective function (i.e. mean absolute
error). From there, we perform an experiment with 100 iterations, visualize the
results, and compare against random search. See
[random_search.ipynb](random_search.ipynb) for some links on troubleshooting
installation setup and a linear version of running an optimization campaign without the
`SelfDrivingLabDemo` class.

## Setup

Package installation and imports

### Package Installation

Assuming you're running this notebook inside of the GitHub repository, do a local
installation (`-e`) in the directory one level about this `notebooks` directory (`../.`)
as follows.
> Note: This only needs to be run once.

In [ ]:
%pip install -e ../.

### Imports

In [1]:
%load_ext autoreload
%autoreload 2 # just some IPython magic to recognize changes to installed packages
import pandas as pd
from self_driving_lab_demo.core import SelfDrivingLabDemo

### SelfDrivingLabDemo

We'll instantiate the class and verify some of the functionality described in the random
search tutorial ([`random_search.ipynb`](random_search.ipynb)).

#### Instantiation

Now, we instantiate the `SelfDrivingLabDemo` class with `autoload=True` so that records
a target to optimize against. This involves selecting a set of target measurements as the "true" input values (i.e. the input
brightness and RGB values that define the target spectrum) based on a random seed,
setting the LED to those values, and then recording the spectrum intensities.

> Note: Instantiating with autoload=True will light the LED briefly.

In [2]:
sdl = SelfDrivingLabDemo(autoload=True)

#### Functionality

We can do similar things to what was done in `random_search.ipynb`. For example, getting
random inputs, observing the sensor data, and evaluating the objective function.

In [3]:
[sdl.get_random_inputs(), sdl.get_random_inputs()]

[(0.27088461699458716, 112, 219, 178), (0.032962071760677336, 249, 194, 200)]

In [4]:
sdl.observe_sensor_data(*sdl.get_random_inputs())

(213, 2029, 2053, 1452, 589, 868, 1372, 695)

In [6]:
sdl.evaluate(*sdl.get_random_inputs())

{'ch415_violet': 581,
 'ch445_indigo': 8492,
 'ch480_blue': 8417,
 'ch515_cyan': 14000,
 'ch560_green': 2823,
 'ch615_yellow': 1325,
 'ch670_orange': 1875,
 'ch720_red': 1232,
 'mae': 2415.25,
 'rmse': 3774.5856063944293}

## Optimization

While there are great numerical tutorials comparing [grid search vs. random search vs.
Bayesian optimization](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46), here, we'll compare these three search methods in a way that perhaps you've never seen before,
namely a self-driving laboratory demo!

### Setup

We define our optimization task parameters and take care of imports.

### Optimization Task Parameters

We'll use 81 iterations repeated 5 times. The use of 81 iterations instead of something
"cleaner" like 50 or 100 is due to constraints of doing uniform (full-factorial) grid
search. $n^d$ number of points are required for uniform grid search, where $n$ and $d$
represent number of points per dimension (`n_pts_per_dim`) and number of dimensions
(`4`), respectively.

In [49]:
num_iter = 2 ** 4
num_repeats = 5
SEEDS = range(10, 10 + num_repeats)

We also instantiate multiple `SelfDrivingLabDemo` instances, each with their own
unique target spectrum.

In [50]:
sdls = [SelfDrivingLabDemo(autoload=True, target_seed=seed) for seed in SEEDS]

Notice that the target_data is different for each.

In [51]:
pd.DataFrame([sdl.target_data for sdl in sdls], columns=sdl.channel_names)

,ch415_violet,ch445_indigo,ch480_blue,ch515_cyan,ch560_green,ch615_yellow,ch670_orange,ch720_red
0,603,3559,6946,21048,3719,3383,6012,1076
1,157,185,434,1990,647,803,1327,489
2,209,891,1005,1383,535,2322,4372,553
3,829,5537,7800,19215,3738,10067,19809,1319
4,959,17321,14654,16970,3359,4872,9151,1648




### Imports

We'll be using `scikit-learn`'s `ParameterGrid` for grid search, `self_driving_lab_demo`'s built-in
`get_random_inputs` for random search, and `ax-platform`'s Gaussian Process Expected
Improvement (GPEI) model for Bayesian
optimization. To help with defining the grid search space, we will also use the
`bounds` and `parameters` class property of `SelfDrivingLabDemo` for convenience.

In [52]:
import numpy as np
from tqdm import trange, tqdm
from sklearn.model_selection import ParameterGrid
from ax import optimize

In [53]:
sdls[0].bounds

{'brightness': [0.0, 0.35], 'R': [0, 255], 'G': [0, 255], 'B': [0, 255]}

In [54]:
sdls[0].parameters

[{'name': 'brightness', 'type': 'range', 'bounds': [0.0, 0.35]},
 {'name': 'R', 'type': 'range', 'bounds': [0, 255]},
 {'name': 'G', 'type': 'range', 'bounds': [0, 255]},
 {'name': 'B', 'type': 'range', 'bounds': [0, 255]}]

### Grid Search

First, we need to define our parameter grid. We'll divide up the 4-dimensional parameter
space as evenly as possible (see `num_pts_per_dim` below).

In [55]:
param_grid = {}
num_pts_per_dim = round(num_iter ** (1 / len(sdl.bounds)))
for name, bnd in sdl.bounds.items():
    param_grid[name] = np.linspace(bnd[0], bnd[1], num=num_pts_per_dim)
    if isinstance(bnd[0], int):
        param_grid[name] = np.round(param_grid[name]).astype(int)
print(f"num_pts_per_dim: {num_pts_per_dim}")

num_pts_per_dim: 2


Notice that there are only 3 distinct values along each dimension.

In [56]:
param_grid

{'brightness': array([0.  , 0.35]),
 'R': array([  0, 255]),
 'G': array([  0, 255]),
 'B': array([  0, 255])}

After assembling the full grid, notice that the total number of points is $3^4 = 81$.

In [57]:
grid = list(ParameterGrid(param_grid))
print("grid:\n", grid[0:4], "...", grid[-1:])
print("\nNumber of grid points: ", len(grid))

grid:
 [{'B': 0, 'G': 0, 'R': 0, 'brightness': 0.0}, {'B': 0, 'G': 0, 'R': 0, 'brightness': 0.35}, {'B': 0, 'G': 0, 'R': 255, 'brightness': 0.0}, {'B': 0, 'G': 0, 'R': 255, 'brightness': 0.35}] ... [{'B': 255, 'G': 255, 'R': 255, 'brightness': 0.35}]

Number of grid points:  16


Now, we can start the actual search. The grid search locations are fixed
for each of the repeat optimization campaigns; however the observed sensor data will be
stochastic and the target spectrum is different for each repeat run. An alternative approach to setting a
fixed budget and varying the target solution would be to see how many iterations it takes to meet a criteria for the
objective function similar to [this post](https://towardsdatascience.com/grid-search-vs-random-search-vs-bayesian-optimization-2e68f57c3c46); however, a fixed budget seems more characteristic of a real chemistry
or materials optimization campaign due to limits on funding, time, and other resources:
(i.e. we'll search until we find what we're looking for, until we run out of
resources, or until we decide it's no longer worth the expense, whichever comes first).

In [58]:
grid_data = [
    [
        sdl.evaluate(pt["brightness"], pt["R"], pt["G"], pt["B"])
        for pt in grid
    ]
    for sdl in tqdm(sdls)
]

100%|██████████| 5/5 [01:07<00:00, 13.43s/it]


### Random Search

Now, let's perform random search as we did before in
[`random_search.ipynb`](random_search.ipynb), storing the inputs and outputs as we go.

In [59]:
%%time
random_inputs = []
random_data = []
for _ in tqdm(range(num_repeats)):
    ri = []
    od = []
    for i in range(num_iter):
        ri.append(sdl.get_random_inputs())
        od.append(sdl.evaluate(*ri[i]))
    random_inputs.append(ri)
    random_data.append(od)

100%|██████████| 5/5 [01:06<00:00, 13.39s/it]

CPU times: user 8.42 s, sys: 4.98 s, total: 13.4 s
Wall time: 1min 6s


### Bayesian Optimization

Ax may run out of memory for low-RAM RPi's such as RPi Zero 2. Even on RPi 4B/RPi
400, it will be slow* for more than about 50 iterations due to the lack of MKL
optimization (a feature of RPi compute hardware).

<p>
<sup>
*i.e. maybe slower than a demo/tutorial should be
</sup>
</p>

In [60]:
%%time
bo_results = []

for sdl in tqdm(sdls):
    def evaluation_function(parameters):
        data = sdl.evaluate(
            parameters["brightness"],
            parameters["R"],
            parameters["G"],
            parameters["B"],
        )
        return data["mae"]

    bo_results.append(optimize(
        parameters=sdl.parameters,
        evaluation_function=evaluation_function,
        minimize=True,
        total_trials = num_iter,
    ))

best_parameters, values, experiment, model = zip(*bo_results)

  0%|          | 0/5 [00:00<?, ?it/s][INFO 08-20 15:58:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter brightness. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-20 15:58:57] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter R. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-20 15:58:57] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter G. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 08-20 15:58:57] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter B. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 's

CPU times: user 4min 21s, sys: 12.9 s, total: 4min 34s
Wall time: 4min 7s


### Analysis

Now that we've run our three optimizations, let's compare the performance in tabular
form and visually.

### Preparing the data

In [61]:
grid_mae = [[g["mae"] for g in gd] for gd in grid_data]
random_mae = [[r["mae"] for r in rd] for rd in random_data]
bayesian_mae = [exp.fetch_data().df["mean"].tolist() for exp in experiment]

In [62]:
mae = np.array([grid_mae, random_mae, bayesian_mae])
mae.shape

(3, 5, 16)

### Tabular

In [73]:
avg_mae = np.mean(np.minimum.accumulate(mae, axis=2), axis=1)
std_mae = np.std(avg_mae, axis=1)
avg_mae.shape

(3, 16)

In [74]:
np.mean(random_mae)

5320.1109375

In [75]:
best_avg_mae = np.min(avg_mae, axis=1)
best_avg_mae

array([2132.1 , 1912.55,  700.35])

### Best Objective vs. Iteration

In [76]:
names = ["grid", "random", "bayesian"]
df = pd.DataFrame({
    **{f"{n}_mae": m for n, m in zip(names, avg_mae)},
    **{f"{n}_std": s for n, s in zip(names, std_mae)},
})


In [77]:
mae_df = pd.melt(df.reset_index(), id_vars=["index"], value_vars = ["grid_mae", "random_mae", "bayesian_mae"], var_name="method", value_name="mae")

std_df = pd.melt(df.reset_index(), id_vars=["index"], value_vars = ["grid_std", "random_std", "bayesian_std"], var_name="method", value_name="std")

mae_df.loc[:, "method"] = mae_df.loc[:, "method"].apply(lambda x: x.replace("_mae", ""))
std_df.loc[:, "method"] = std_df.loc[:, "method"].apply(lambda x: x.replace("_std", ""))

In [78]:
results_df = mae_df.merge(std_df, on=["method", "index"]).rename(columns=dict(index="iteration"))
results_df

,iteration,method,mae,std
0,0,grid,4803.500,1042.922493
1,1,grid,4802.750,1042.922493
2,2,grid,4802.575,1042.922493
3,3,grid,4212.650,1042.922493
4,4,grid,4211.250,1042.922493
5,5,grid,3042.525,1042.922493
6,6,grid,3042.525,1042.922493
7,7,grid,2547.800,1042.922493
8,8,grid,2547.800,1042.922493
9,9,grid,2263.100,1042.922493


### Visualization
As we might expect, Bayesian optimization outperforms random search while grid and
random search are on par with each other.

In [89]:
# import plotly.express as px
from self_driving_lab_demo.utils.plotting import line

fig = line(
    data_frame=results_df,
    x="iteration",
    y="mae",
    error_y="std",
    error_y_mode="band",
    color="method",
)

fig

## Repeat Measurement Stochasticity

In [80]:
sdls[0].get_target_inputs()

(0.33460059837014133, 53, 211, 38)

In [81]:
sdls[0].target_data

(603, 3559, 6946, 21048, 3719, 3383, 6012, 1076)

In [83]:
check_data = []
from time import sleep
for _ in range(10):
    check_data.append(sdls[0].evaluate(*sdls[0].get_target_inputs()))

In [84]:
pd.DataFrame(check_data).agg([np.mean, np.std]).T

,mean,std
ch415_violet,619.800000,0.788811
ch445_indigo,3569.300000,3.400980
ch480_blue,6962.200000,1.932184
ch515_cyan,21046.500000,12.843070
ch560_green,3750.400000,2.590581
ch615_yellow,3406.700000,6.815831
ch670_orange,6074.000000,4.189935
ch720_red,1189.900000,6.590397
mae,35.550000,2.287648
rmse,49.028371,2.716991


## Code Graveyard

In [ ]:
# for _ in trange(num_repeats):
#     for params in tqdm(grid):
#         sdl.evaluate(*params)

# pd.concat((mae_df, std_df), axis=1, join="inner")

# best_parameters = []
# values = []
# experiment = []
# model = []

    # best_parameters.append(bp)
    # values.append(v)
    # experiment.append(exp)
    # model.append(m)